# Report of UCI Breast Cancer Wisconsin (Diagnostic) Dataset

## Data Source


This database is also available through the UW CS ftp server: 
ftp ftp.cs.wisc.edu cd math-prog/cpo-dataset/machine-learn/WDBC/

Also can be found on UCI Machine Learning Repository:

https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29

Or this dataset can be found in Kaggle(Where I found this dataset):

https://www.kaggle.com/uciml/breast-cancer-wisconsin-data

## Data Description

**Missing value and classfication:**

Missing attribute values: none

Class distribution: 357 benign, 212 malignant

## Preprocessing
First, import dataset into pands which has dataframe to show the shape and form of dataset.

In [1]:
import numpy as np
import pandas as pd
from IPython.display import display, HTML
# read dataset from download file
dataname = ['id','diagnosis','radius_mean','texture_mean','perimeter_mean',
            'area_mean','smoothness_mean','compactness_mean','concavity_mean',
            'concave points_mean','symmetry_mean','fractal_dimension_mean','radius_se',
            'texture_se','perimeter_se','area_se','smoothness_se','compactness_se',
            'concavity_se','concave points_se','symmetry_se','fractal_dimension_se',
            'radius_worst','texture_worst','perimeter_worst','area_worst','smoothness_worst',
            'compactness_worst','concavity_worst','concave points_worst','symmetry_worst',
            'fractal_dimension_worst']

data = pd.read_csv('data.csv', header = 0)
data.shape
#display(data)
#HTML(data.to_html())

from IPython.display import display
data = data.drop("Unnamed: 32", axis =1 )
display(data.head())

nddata = data
d = {'M' : 0, 'B' : 1}
nddata['diagnosis'] =nddata['diagnosis'].map(d)

xnddata = nddata

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


## Getting Tensorflow!

Although all those methods have already give us accuracy more than 90%, it is still not high enough for predicting cancer since this disease is dangerous. I will try to apply my method to by using deep learning of powerful Tensorflow!

In [2]:
nddata = xnddata[0:469]
#nddata.shape
#display(nddata)

trnddata = xnddata[469:569]
#trnddata.shape
#display(trnddata)

In [3]:
import tensorflow as tf
sess = tf.InteractiveSession()


x = tf.placeholder("float", [100, 30])
y_ = tf.placeholder("float", [100, 1])
W = tf.Variable(tf.zeros([30, 1]))
b = tf.Variable(tf.zeros([1,1]))

y = tf.nn.softmax(tf.matmul(x, W) + b)

loss = tf.reduce_mean(tf.square(abs(y - y_)))
optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

writer = tf.train.SummaryWriter('./P5tuning', sess.graph)

In [4]:
sess.run(tf.initialize_all_variables())

In [5]:
for i in range(1000):
    tfdata = nddata.sample(frac=1)
    ntfdata = tfdata[dataname[1:32]]
    
    
    traindata = ntfdata[:100]
    beforetraindata = traindata[dataname[2:32]]
    finaltraindata = (beforetraindata - beforetraindata.mean())/(beforetraindata.max() - beforetraindata.min())

    x_train = np.float32(finaltraindata[dataname[2:32]])
    y_train = np.float32(traindata[dataname[1]])
    x_test = np.reshape(x_train, (100, 30))
    y_test = np.reshape(y_train, (100, 1))
    sess.run(optimizer, feed_dict={x:x_test, y_:y_test})
    

In [6]:
correct_production = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_production, "float"))

traindata = trnddata
beforetraindata =  trnddata[dataname[2:32]]
finaltraindata = (beforetraindata - beforetraindata.mean())/(beforetraindata.max() - beforetraindata.min())

x_train = np.float32(finaltraindata[dataname[2:32]])
y_train = np.float32(traindata[dataname[1]])
x_test = np.reshape(x_train, (100, 30))
y_test = np.reshape(y_train, (100, 1))

print(accuracy.eval(feed_dict={x: x_test, y_: y_test}))

1.0


## CNN

In [7]:
import numpy as np
import pandas as pd
from IPython.display import display, HTML
# read dataset from download file
dataname = ['id','diagnosis','radius_mean','texture_mean','perimeter_mean',
            'area_mean','smoothness_mean','compactness_mean','concavity_mean',
            'concave points_mean','symmetry_mean','fractal_dimension_mean','radius_se',
            'texture_se','perimeter_se','area_se','smoothness_se','compactness_se',
            'concavity_se','concave points_se','symmetry_se','fractal_dimension_se',
            'radius_worst','texture_worst','perimeter_worst','area_worst','smoothness_worst',
            'compactness_worst','concavity_worst','concave points_worst','symmetry_worst',
            'fractal_dimension_worst']

data = pd.read_csv('data.csv', header = 0)
data.shape
#display(data)
#HTML(data.to_html())

from IPython.display import display
data = data.drop("Unnamed: 32", axis =1 )
display(data.head())

nddata = data
d = {'M' : 0, 'B' : 1}
nddata['diagnosis'] =nddata['diagnosis'].map(d)

xnddata = nddata

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [8]:
import tensorflow as tf
sess = tf.InteractiveSession()

nddata = xnddata[0:500]
trnddata = xnddata[500:569]

In [9]:
x = tf.placeholder("float", [30, 30])
y_ = tf.placeholder("float", [30, 1])
W = tf.Variable(tf.zeros([30, 1]))
b = tf.Variable(tf.zeros([1,1]))

sess.run(tf.initialize_all_variables())
y = tf.matmul(x,W) + b
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, y_))
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)

In [11]:
for i in range(1000):
    tfdata = nddata.sample(frac=1)
    ntfdata = tfdata[dataname[1:32]]
    
    traindata = ntfdata[:30]
    beforetraindata = traindata[dataname[2:32]]
    finaltraindata = (beforetraindata - beforetraindata.mean())/(beforetraindata.max() - beforetraindata.min())

    x_train = np.float32(finaltraindata[dataname[2:32]])
    y_train = np.float32(traindata[dataname[1]])
    x_test = np.reshape(x_train, (30, 30))
    y_test = np.reshape(y_train, (30, 1))
    sess.run(train_step, feed_dict={x:x_test, y_:y_test})

In [12]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
xtraindata = trnddata.sample(frac=1)

traindata =  xtraindata[:30]
beforetraindata = traindata[dataname[2:32]]
finaltraindata = (beforetraindata - beforetraindata.mean())/(beforetraindata.max() - beforetraindata.min())


x_train = np.float32(finaltraindata[dataname[2:32]])
y_train = np.float32(traindata[dataname[1]])
x_test = np.reshape(x_train, (30, 30))
y_test = np.reshape(y_train, (30, 1))
print(accuracy.eval(feed_dict={x: x_test, y_: y_test}))

1.0


In [13]:
def weight_varible(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_varible(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding = 'SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding = 'SAME')    

In [14]:
W_conv1 = weight_varible([5,5,1,30])
b_conv1 = bias_varible([30])
x_image = tf.reshape(x, [-1, 30, 30, 1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1)+b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_varible([5,5,30,60])
b_conv2 = bias_varible([60])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2)+b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [16]:
W_fc1 = weight_varible([8*8*60, 1024])
b_fc1 = bias_varible([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 8*8*60])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_varible([1024,10])
b_fc2 = bias_varible([10])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [18]:
#cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
#cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
#correct_prediction2 = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
correct_pre = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))                                                                       
accuracy = tf.reduce_mean(tf.cast(correct_pre, tf.float32))
sess.run(tf.initialize_all_variables())
for i in range(2001):
    tfdata = nddata.sample(frac=1)
    ntfdata = tfdata[dataname[1:32]]
    
    traindata = ntfdata[:30]
    beforetraindata = traindata[dataname[2:32]]
    finaltraindata = (beforetraindata - beforetraindata.mean())/(beforetraindata.max() - beforetraindata.min())

    x_train = np.float32(finaltraindata[dataname[2:32]])
    y_train = np.float32(traindata[dataname[1]])
    x_test = np.reshape(x_train, (30, 30))
    y_test = np.reshape(y_train, (30, 1))
    if i%200 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:x_test, y_: y_test, keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
     
    
    sess.run(train_step, feed_dict={x:x_test, y_: y_test, keep_prob: 0.5})

ValueError: Dimension 0 in both shapes must be equal, but are 1 and 30

In [46]:
xtraindata = trnddata.sample(frac=1)

traindata =  xtraindata[:30]
beforetraindata = traindata[dataname[2:32]]
finaltraindata = (beforetraindata - beforetraindata.mean())/(beforetraindata.max() - beforetraindata.min())

x_train = np.float32(finaltraindata[dataname[2:32]])
y_train = np.float32(traindata[dataname[1]])
x_test = np.reshape(x_train, (30, 30))
y_test = np.reshape(y_train, (30, 1))
print("test accuracy %g"%accuracy.eval(feed_dict={x:x_test, y_: y_test, keep_prob: 1.0}))

test accuracy 0
